# 누락된 데이터 다루기

### 누락된 값 식별

In [1]:
import pandas as pd
from io import StringIO
csv_data = \
'''A,B,C,D
1.0, 2.0, 3.0, 4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [2]:
#누락된 값의 갯수
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

In [3]:
#누락된 값이 있는 행 제거
df.dropna(axis=0)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [4]:
#누락된 값이 있는 열 제거
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [5]:
#모든 값이 NaN인 행 삭제
df.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [6]:
#실수 값이 4 개보다 작은 행 삭제
df.dropna(thresh=4)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [7]:
#특정 열에 NaN이 있는 행만 삭제
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


##### 누락된 값 대체 보간기법
디폴트 : NaN값을 열의 평균으로 만들어줌

In [8]:
import numpy as np
from sklearn.impute import SimpleImputer
simr = SimpleImputer(missing_values=np.nan, strategy='mean')
simr =simr.fit(df.values)
imputed_data = simr.transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [9]:
#NaNa값을 행의 평균으로 만들기
import numpy as np
from sklearn.preprocessing import FunctionTransformer
ftr_simr = FunctionTransformer(lambda X : simr.fit_transform(X.T).T,validate=False)
imputed_data = ftr_simr.fit_transform(df.values)
imputed_data

array([[ 1.        ,  2.        ,  3.        ,  4.        ],
       [ 5.        ,  6.        ,  6.33333333,  8.        ],
       [10.        , 11.        , 12.        , 11.        ]])

### 범주형 데이터

In [10]:
import pandas as pd
df = pd.DataFrame([
    ['green', 'M', 10.1, 'class1'],
    ['red', 'L',13.5,'class2'],
    ['blue', 'XL',15.3,'class1']])
df.columns = ['color', 'size','price','classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


가정 : XL = L + 1 = M + 2

In [11]:
size_mapping = { 
'XL' : 3,
'L' : 2,
'M' : 1}
df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [12]:
#반대로 매핑하기
inv_size_mapping ={v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

##### 클래스 레이블인코딩

In [13]:
import numpy as np
class_mapping = {label : idx for idx, label in
                 enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

In [14]:
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


In [15]:
inv_class_mapping = {v : k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [16]:
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y


array([0, 1, 0])

In [17]:
class_le.inverse_transform(y)

array(['class1', 'class2', 'class1'], dtype=object)

##### 순서가 없는 특성에 원-핫 인코딩 적용


In [18]:
X = df[['color','size','price']].values
color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:,0])
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

열마다 다른 변환을 적용 (sklearn)

In [19]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder(dtype = np.int)
col_trans = ColumnTransformer([('ord_enc', ord_enc, ['color','classlabel'])])
X_trans = col_trans.fit_transform(df)
X_trans

array([[1, 0],
       [2, 1],
       [0, 0]])

In [20]:
col_trans.named_transformers_['ord_enc'].inverse_transform(X_trans)

array([['green', 'class1'],
       ['red', 'class2'],
       ['blue', 'class1']], dtype=object)

In [21]:
from sklearn.preprocessing import OneHotEncoder
oh_enc = OneHotEncoder(categories='auto')
col_trans = ColumnTransformer([('oh_enc', oh_enc,[0])], remainder = 'passthrough')
col_trans.fit_transform(X)


array([[0.0, 1.0, 0.0, 1, 10.1],
       [0.0, 0.0, 1.0, 2, 13.5],
       [1.0, 0.0, 0.0, 3, 15.3]], dtype=object)

In [22]:
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

In [23]:
#문자열만 원핫 인코딩함
pd.get_dummies(df[['price', 'color','size']])

,price,size,color_blue,color_green,color_red
0,10.1,1,0,1,0
1,13.5,2,0,0,1
2,15.3,3,1,0,0


In [24]:
pd.get_dummies(df[['price', 'color','size']],
              drop_first=True)

,price,size,color_green,color_red
0,10.1,1,1,0
1,13.5,2,0,1
2,15.3,3,0,0


In [25]:
#슬라이싱
from sklearn.preprocessing import OneHotEncoder
oh_enc = OneHotEncoder(categories='auto')
col_trans = ColumnTransformer([('oh_enc', oh_enc,[0])], remainder = 'passthrough')
col_trans.fit_transform(X)[:,1:]

array([[1.0, 0.0, 1, 10.1],
       [0.0, 1.0, 2, 13.5],
       [0.0, 0.0, 3, 15.3]], dtype=object)

### 데이터셋을 훈련 세트와 테스트 세트로 나누기


In [27]:
df_wine = pd.read_csv('https://archive.ics.uci.edu/'
                     'ml/machine-learning-databases/'
                     'wine/wine.data', header=None)
df_wine.columns = ['Class label','Alcohol','Malic acid',
                  'Ash','Alcalinity of ash','Magnesium',
                  'Total phenols','Flavanoids',
                  'Nonflavanoid phenols',
                  'Proanthocyanins','Colro intensity', 'Hue','OD280/OD315 of diluted wines',
                  'Proline']
print('클래스 레이블',np.unique(df_wine['Class label']))
df_wine.head()

클래스 레이블 [1 2 3]


,Class label,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Colro intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


### 특성 스케일 맞추기
정규화, 표준화   
정규화 : 최소 값과 최대값을 사용하여 값을 0~1사이의 값으로 변환    
표준화 : 평균과 표준편차를 사용하여 값을 정규분포에 근사하는 값들의 분포로 바꿔줌.   

표준화가 정규화에비해 이상치에 덜 민감함.

In [35]:
#표준화, 정규화 예시
ex = np.array([0,1,2,3,4,5])
print('표준화:' , (ex - ex.mean()) / ex.std())

print('정규화:', (ex - ex.min()) / (ex.max() -ex.min()))

표준화: [-1.46385011 -0.87831007 -0.29277002  0.29277002  0.87831007  1.46385011]
정규화: [0.  0.2 0.4 0.6 0.8 1. ]


In [36]:
#최소 - 최대 스케일 변환(정규화)
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X_train_norm = mms.fit_transform(X_train)
X_test_norm = mms.transform(X_test)

In [37]:
#표준화
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

In [38]:
from sklearn.linear_model import LogisticRegression
LogisticRegression(solver = 'liblinear', multi_class='auto', penalty='l1')

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [42]:
lr = LogisticRegression(solver = 'liblinear', multi_class='auto',
                        penalty ='l1',C=1.0, random_state=42)
lr.fit(X_train_std, y_train)
print('훈련 정확도 :' ,lr.score(X_train_std, y_train))
print('테스트 정확도 : ' , lr.score(X_test_std, y_test))


훈련 정확도 : 1.0
테스트 정확도 :  1.0


In [43]:
lr.intercept_

array([-1.26385366, -1.21585801, -2.36829063])

In [44]:
lr.coef_

array([[ 1.2460319 ,  0.1807062 ,  0.74653137, -1.16381009,  0.        ,
         0.        ,  1.15973686,  0.        ,  0.        ,  0.        ,
         0.        ,  0.55705673,  2.50870623],
       [-1.53734059, -0.38713982, -0.99508471,  0.36483879, -0.05969195,
         0.        ,  0.66803946,  0.        ,  0.        , -1.93424688,
         1.23402627,  0.        , -2.23196901],
       [ 0.13553583,  0.1672072 ,  0.35653325,  0.        ,  0.        ,
         0.        , -2.44311777,  0.        ,  0.        ,  1.56723369,
        -0.82103411, -0.48576419,  0.        ]])

규제 파라미터 C를  변화함에 따라 속성들의 가중 치를 알 수 있다.

##### 순차 특성 선택 알고리즘
차원축소 ; 특성 선택, 특성 추출.   
전통적인 순차 특성 선택 알고리즘 : 순차 후진 선택(SBS).   
그외, 랜덤포레스트 특성중요도